<a href="https://colab.research.google.com/github/EoghanPower/pr/blob/main/Create_a_TweetsDataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#Creating a Data Frame of Tweets on S&P 500 Companies

In [2]:
# import libraries
import tweepy, json 
from textblob import TextBlob 
from wordcloud import wordcloud
import pandas as pd
import numpy as np 
import re
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import webbrowser
import time 
import requests
import requests_oauthlib
import os

In [3]:
#authentication store 0auth credentials in variables
consumer_key = 'eHrpqTZVq5bfGdjC8juN7fSCs'
consumer_secret = 'itfojeERaMFXtwQFI1emQNT3vwCHs414OJdOXlpN2RaBAw3MIz'
AccessToken = '264845011-WhC9gKJlC3TayNsXg4ViBIuwaK0D5Sof3lxPTZeN'
AccessTokenSecret = 'dE5difJJmYP5UYllqeQwTsovcoXyvco9Ww2QMxiR365D3'

In [4]:
#Pass 0auth details to tweepy's 0auth handlet 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(AccessToken, AccessTokenSecret)

In [5]:
##SetAPIobject
api = tweepy.API(auth, wait_on_rate_limit= True)

In [6]:
#StreamListener
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets.txt", "w")

    def on_status(self, status):
        tweet = status._json
        self.file.write( json.dumps(tweet) + '\n' )
        self.num_tweets += 1
        if self.num_tweets < 10001:
            return True
        else:
            return False
        self.file.close()

    def on_error(self, status):
        print(status)

In [7]:
#initialize stream listener
l = MyStreamListener()
#Create stream object with authentication
stream = tweepy.Stream(auth, l)
#Filter Twitter Streams to capture data by the keywords:
stream.filter(languages = ['en'],track=['Apple', 'Google', 'Microsoft','Amazon',  'Facebook', 'JPMorgan',   'Johnson and Johnson', '  Exxon', 'Bank of America', 'Wal-Mart, Wells Fargo', 'Berkshire Hathaway', 'AT&T ', 'Tesla', 'Gamestop' ])

In [8]:
#import package
import json

#String of path to file: tweets_data_path
tweets_data_path = 'tweets.txt'

#initialize empty list to store tweets: tweets_data
tweets_data = []

In [9]:
#open connection to file
tweets_file = open(tweets_data_path, 'r')

In [10]:
# Read in tweets and store in list: tweets_data
for line in tweets_file:
  tweet = json.loads(line)
  tweets_data.append(tweet)

In [11]:
#close connection to file
tweets_file.close()


In [12]:
print(tweets_data[0].keys())

dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])


In [13]:
# Build DataFrame of tweet texts and languages
df = pd.DataFrame(tweets_data, columns= ['text'])

In [14]:
print(df.head(10))

                                                text
0  My allies have called them all far worse.\nI'm...
1  @ThatNeighborBoy Idk about y'all but I definit...
2  RT @apenftorg: #APENFT Christmas Red Apple Eve...
3  Yaaayy! A massive milestone - a few years back...
4  RT @1520cart: [ early christmas giveaway ]\n\n...
5  RT @ATINGlobalSoc: #LoveYours by Ohwon Lee ft....
6  RT @DarrenF_Jones: In Episode 2 of our comedy ...
7  RT @LauraPh27961838: Thanks Boris Johnson for ...
8  RT @amazing_physics: “If you want to find the ...
9  RT @CaucasianJames: mom just updated her cover...


In [15]:
#clean the tweets
def cleanTwt(twt): 
  twt = re.sub('#apple','Apple', twt)
  twt = re.sub('#google','Google', twt) 
  twt = re.sub('#microsoft','Microsoft', twt) 
  twt = re.sub('#amazon','Amazon', twt) 
  twt = re.sub('#facebook','Facebook', twt) 
  twt = re.sub('#JPMorgan','JPMorgan', twt) 
  twt = re.sub('#JohnsonandJohnson','Johnson and Johnson', twt) 
  twt = re.sub('#Exxon','Exxon', twt) 
  twt = re.sub('#BankofAmerica','Bank of America', twt) 
  twt = re.sub('#Wal-Mart','Wal-Mart', twt) 
  twt = re.sub('#Wells Fargo','Wells Fargo', twt) 
  twt = re.sub('#BerkshireHathaway','Berkshire Hathaway', twt) 
  twt = re.sub('#Gamestop','Gamestop', twt) 
  twt = re.sub('#Tesla ','Tesla', twt)#removes the # 
  twt =re.sub('#[A-Za-z0-9]+', '',twt) #removes any strings with a # 
  twt = re.sub('\\n', '', twt) #removes the '\n' string
  twt = re.sub('https?:\/\/\S+', '', twt) #removes any hyperlinks
  twt = re.sub('RT', '', twt) #cleans retweets
  twt = re.sub('@[A-Za-z0-9]+', ',', twt)#Removes tagged accounts
  return twt

In [16]:
#clean them using the above function

In [17]:
df['Cleaned_Tweets'] = df['text'].apply(cleanTwt)

In [18]:
#show the set 
df.head(10)

,text,Cleaned_Tweets
0,My allies have called them all far worse.\nI'm...,My allies have called them all far worse.I'm s...
1,@ThatNeighborBoy Idk about y'all but I definit...,", Idk about y'all but I definitely trust the D..."
2,RT @apenftorg: #APENFT Christmas Red Apple Eve...,",: Christmas Red Apple Event （72 hours）Follo..."
3,Yaaayy! A massive milestone - a few years back...,Yaaayy! A massive milestone - a few years back...
4,RT @1520cart: [ early christmas giveaway ]\n\n...,",: [ early christmas giveaway ]1 winner of 10..."
5,RT @ATINGlobalSoc: #LoveYours by Ohwon Lee ft....,",: by Ohwon Lee ft. SB19 is now on Bugs!Bugs..."
6,RT @DarrenF_Jones: In Episode 2 of our comedy ...,",_Jones: In Episode 2 of our comedy podcast d..."
7,RT @LauraPh27961838: Thanks Boris Johnson for ...,",: Thanks Boris Johnson for your abuse and ne..."
8,RT @amazing_physics: “If you want to find the ...,",_physics: “If you want to find the secrets o..."
9,RT @CaucasianJames: mom just updated her cover...,",: mom just updated her cover photo on facebo..."


In [19]:
#create a function to get subjectivity
def getSubjectivity(twt):
  return TextBlob(twt).sentiment.subjectivity
#create a function to get polarity
def getPolarity(twt):
  return TextBlob(twt).sentiment.polarity


In [20]:
#create two new columns called subjectivity and polarity
df['Subjectivity'] = df['Cleaned_Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Cleaned_Tweets'].apply(getPolarity)

In [21]:
#show data
df.head(20)

,text,Cleaned_Tweets,Subjectivity,Polarity
0,My allies have called them all far worse.\nI'm...,My allies have called them all far worse.I'm s...,0.962963,-0.133333
1,@ThatNeighborBoy Idk about y'all but I definit...,", Idk about y'all but I definitely trust the D...",0.500000,0.000000
2,RT @apenftorg: #APENFT Christmas Red Apple Eve...,",: Christmas Red Apple Event （72 hours）Follo...",0.000000,0.000000
3,Yaaayy! A massive milestone - a few years back...,Yaaayy! A massive milestone - a few years back...,0.366667,-0.066667
4,RT @1520cart: [ early christmas giveaway ]\n\n...,",: [ early christmas giveaway ]1 winner of 10...",0.650000,-0.312500
5,RT @ATINGlobalSoc: #LoveYours by Ohwon Lee ft....,",: by Ohwon Lee ft. SB19 is now on Bugs!Bugs...",0.400000,0.000000
6,RT @DarrenF_Jones: In Episode 2 of our comedy ...,",_Jones: In Episode 2 of our comedy podcast d...",0.000000,0.000000
7,RT @LauraPh27961838: Thanks Boris Johnson for ...,",: Thanks Boris Johnson for your abuse and ne...",0.200000,0.200000
8,RT @amazing_physics: “If you want to find the ...,",_physics: “If you want to find the secrets o...",0.000000,0.000000
9,RT @CaucasianJames: mom just updated her cover...,",: mom just updated her cover photo on facebo...",0.000000,0.000000


In [22]:
#Create a function to get the sentiment text
def getSentiment(score):
  if score <0:
    return 'Negative'
  elif score == 0:
      return 'Neutral'
  else:
      return 'Positive'

In [23]:
#Create column to store the text sentiment
df['Sentiment'] = df['Polarity'].apply(getSentiment)
#Show the data
df.head(7)

,text,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,My allies have called them all far worse.\nI'm...,My allies have called them all far worse.I'm s...,0.962963,-0.133333,Negative
1,@ThatNeighborBoy Idk about y'all but I definit...,", Idk about y'all but I definitely trust the D...",0.500000,0.000000,Neutral
2,RT @apenftorg: #APENFT Christmas Red Apple Eve...,",: Christmas Red Apple Event （72 hours）Follo...",0.000000,0.000000,Neutral
3,Yaaayy! A massive milestone - a few years back...,Yaaayy! A massive milestone - a few years back...,0.366667,-0.066667,Negative
4,RT @1520cart: [ early christmas giveaway ]\n\n...,",: [ early christmas giveaway ]1 winner of 10...",0.650000,-0.312500,Negative
5,RT @ATINGlobalSoc: #LoveYours by Ohwon Lee ft....,",: by Ohwon Lee ft. SB19 is now on Bugs!Bugs...",0.400000,0.000000,Neutral
6,RT @DarrenF_Jones: In Episode 2 of our comedy ...,",_Jones: In Episode 2 of our comedy podcast d...",0.000000,0.000000,Neutral


In [24]:
df.drop(columns = 'text')

,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,My allies have called them all far worse.I'm s...,0.962963,-0.133333,Negative
1,", Idk about y'all but I definitely trust the D...",0.500000,0.000000,Neutral
2,",: Christmas Red Apple Event （72 hours）Follo...",0.000000,0.000000,Neutral
3,Yaaayy! A massive milestone - a few years back...,0.366667,-0.066667,Negative
4,",: [ early christmas giveaway ]1 winner of 10...",0.650000,-0.312500,Negative
...,...,...,...,...
9995,🚨🚨 émon : Celebrations Collections - RESTOCK🚨🚨,0.000000,0.000000,Neutral
9996,LIVE: Midfield Action | Host: Joe Laka | 22/12...,0.300000,0.118182,Positive
9997,",: Be Swept off your feet and Spend your Nigh...",0.000000,0.000000,Neutral
9998,",: UPDATE: PNP Health Service recorded the fo...",0.616667,0.133333,Positive


In [25]:
#drop columns to make it easier to read
df.head()

,text,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,My allies have called them all far worse.\nI'm...,My allies have called them all far worse.I'm s...,0.962963,-0.133333,Negative
1,@ThatNeighborBoy Idk about y'all but I definit...,", Idk about y'all but I definitely trust the D...",0.500000,0.000000,Neutral
2,RT @apenftorg: #APENFT Christmas Red Apple Eve...,",: Christmas Red Apple Event （72 hours）Follo...",0.000000,0.000000,Neutral
3,Yaaayy! A massive milestone - a few years back...,Yaaayy! A massive milestone - a few years back...,0.366667,-0.066667,Negative
4,RT @1520cart: [ early christmas giveaway ]\n\n...,",: [ early christmas giveaway ]1 winner of 10...",0.650000,-0.312500,Negative


In [26]:
df.tail()

,text,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
9995,🚨🚨 #RESTOCK #Pokémon #TCG: Celebrations Collec...,🚨🚨 émon : Celebrations Collections - RESTOCK🚨🚨,0.000000,0.000000,Neutral
9996,LIVE: Midfield Action | Host: Joe Laka | 22/12...,LIVE: Midfield Action | Host: Joe Laka | 22/12...,0.300000,0.118182,Positive
9997,RT @RaelleLogan1: Be Swept off your feet and S...,",: Be Swept off your feet and Spend your Nigh...",0.000000,0.000000,Neutral
9998,RT @pnppio: UPDATE: PNP Health Service recorde...,",: UPDATE: PNP Health Service recorded the fo...",0.616667,0.133333,Positive
9999,News: The Jazz aren't looking at wins and los...,News: The Jazz aren't looking at wins and los...,0.200000,0.300000,Positive


In [27]:
#export the dataframe to sheets
#export to google sheets part one authentication
!pip install gspread
from google.colab import auth as au
au.authenticate_user()

In [28]:
!pip install df2gspread

     |████████████████████████████████| 56 kB 3.1 MB/s 
  Created wheel for df2gspread: filename=df2gspread-1.0.4-py3-none-any.whl size=11952 sha256=bbc403eba8bd87b3487ed80e7de0fc2b67475252de17f63f0a42efb864fdb6e8
  Stored in directory: /root/.cache/pip/wheels/4d/78/20/cf9a2810f93885ad71bd351c8b604455ba160ac4b0e9b7d526
Successfully built df2gspread
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 4.4.0 requires google-api-python-client>=1.7.8, but you have google-api-python-client 1.6.7 which is incompatible.
earthengine-api 0.1.290 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.6.7 whi

In [29]:
import gspread
from oauth2client.client import GoogleCredentials

In [30]:
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [31]:
from gspread.models import Worksheet
#export to google sheets part two export
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [32]:
from df2gspread import df2gspread as d2g

In [33]:
spreadsheet_key = '1IjzY-rTedrKHQNz9PahAeCkqZ3zF-Z6h-GiMw8zysNY'
wks_name = 'Master'
d2g.upload(df, spreadsheet_key, wks_name, credentials= GoogleCredentials.get_application_default(), row_names=True)

<Worksheet 'Master' id:1124026790>